# Item:
- propteries: [id, name, price]
- methods: getter + setter, toString, compareTo

In [38]:
class Item:
    def __init__(self, id: int, name: str, price: float):
        self.__id = id
        self.__name = name
        self.__price = price

    @property
    def id(self) -> int:
        return self.__id

    @property
    def name(self) -> str:
        return self.__name

    @property
    def price(self) -> float:
        return self.__price
    
    @id.setter
    def id(self, id: int):
        self.__id = id
  
    @name.setter
    def name(self, name: str):
        self.__name = name

    @price.setter
    def price(self, price: float):
        self.__price = price  
        
    def __str__ (self):
        return f"Item(id={self.__id}, name='{self.__name}', price={self.__price})" 
    
    # so sanh 2 item 
    def __lt__ (self, other):
        # kiem tra other la thuc the cua Item
        if isinstance(other, Item):
            return self.__id < other.__id
        return NotImplemented
    
    def compare_to(self, other):
        if not isinstance(other, Item):
            raise ValueError("Can only compare with another Item!")
        return self.__id - other.__id

# Account: 
- properties: [id, name, score]
- methods: getLoyaltyPromo() - 15%

In [39]:
class Account:
    def __init__(self, id: int, name: str):
        self.id = id
        self.name = name
        self.__score = 0
        
    @property
    def score(self):
        return self.__score
    
    def add_score(self, points: int):
        self.__score += points

    def getLoyaltyPromo(self):
        """15% discount if score is high (>= 100), else no discount"""
        return 0.15 if self.__score >= 100 else 0.0

    def __str__(self):
        return f"Account(id={self.id}, name='{self.name}', score={self.__score})"

# Order:
- properties: [id, account, item_list (dict), voucher (str)]
- methods: getVoucherPromo() - 10%, getBulkOrderPromo() - 10%, setScore(), printBill(), total()

In [40]:
# Item Manager: class co chua danh sach mon hang duoc mua
class ItemManager:
    def __init__(self):
        # khi khai bao cho mac dinh danh sach mon hang la 0
        self.items = {}

    def add_item(self, item):
        # (item obj, qty)
        if item.id not in self.items:
            self.items[item.id] = (item, 1)
        else:
            new_qty = self.items[item.id][1] + 1
            self.items[item.id] = (item, new_qty)

    def edit_item(self, id, new_item: Item):
        if id in self.items:
            self.items[id] = (new_item, self.items[id][1])
        else:
            raise Exception("Item not found")

    def remove_item(self, id):
        if id in self.items:
            del self.items[id]
        else:
            raise Exception("Item not found")

    def get_price_by_qty(self, id):
        if id in self.items:
            return self.items[id][0].price * self.items[id][1]
        else:
            raise Exception("Item not found")

    def get_sum_qty(self):
        return sum(item[1] for item in self.items.values())
    
    def get_total_of_list(self):
        return sum(item[1] * item[0].price for item in self.items.values())
    
    def __str__(self):
        return "\n".join(
            f"{item} | Quantity: {qty}" for item, qty in self.items.values()
        )

In [41]:
item1 = Item(1, "Milk", 25000)
item2 = Item(2, "Bread", 15000)

manager = ItemManager()
manager.add_item(item1)
manager.add_item(item1)
manager.add_item(item2)

print(manager)
print("Total quantity:", manager.get_sum_qty())
print("Total price:", manager.get_total_of_list())

Item(id=1, name='Milk', price=25000) | Quantity: 2
Item(id=2, name='Bread', price=15000) | Quantity: 1
Total quantity: 3
Total price: 65000


In [42]:
class Order:
    def __init__(
        self, account: Account, order_id: str, item_list: ItemManager, voucher: str = ""
    ):
        self.account = account
        self.order_id = order_id
        self.item_list = item_list
        self.voucher = voucher

    # tinh tong + voucher + account score + soluonglon
    def calculate_total(self):
        soluonglon = 0.1 if self.item_list.get_sum_qty() >= 10 else 0
        khachhangthanthiet = self.account.getLoyaltyPromo()
        voucher = 0.1 if self.voucher != "" else 0
        tong = self.item_list.get_total_of_list() * (
            1 - soluonglon - khachhangthanthiet - voucher
        )
        self.__set_new_score(tong)
        return tong

    def __set_new_score(self, bill_total):
        new_point = int(bill_total * 0.1)
        self.account.add_score(new_point)

    # in bill (luu vao file txt: bill_<order_id>.txt)
    def printBill(self):
        total_cost = self.calculate_total()

        with open(f"bill_{self.order_id}.txt", "w") as file:
            file.write(f"Bill ID: {self.order_id}\n")
            file.write(f"Voucher: {self.voucher}\n")
            file.write(f"Loyalty Discount: {self.account.getLoyaltyPromo() * 100}%\n")
            file.write(
                f"Bulk Order Discount: {0.1 if self.item_list.get_sum_qty() >= 10 else 0  * 100}%\n"
            )
            file.write(f"Total (after discounts): ${total_cost:.2f}\n")
            file.write(f"Score (after order): {self.account.score}\n")
            file.write("\nItems:\n")
            file.writelines(self.item_list.__str__())
        print("Done!")

In [43]:
acc1 = Account(1, "Diep")
acc2 = Account(2, "Nguyen")

ord1 = Order(account=acc1, order_id="001", item_list=manager, voucher="ABC")
ord2 = Order(account=acc1, order_id="002", item_list=manager, voucher="")
ord1.printBill()
ord2.printBill()

Done!
Done!
